In [1]:
import pandas as pd 
import geopandas as gpd 
from shapely.geometry import Point
import numpy as np 

In [2]:
# Load Wilayas Features and calculate polygon centroids
wilayas_gjson = '../dz-admin/wilayas_48.geojson'
wilayas_gdf = gpd.read_file(wilayas_gjson)
wilayas_gdf = wilayas_gdf[['code', 'nom', 'geometry']]
wilayas_gdf = wilayas_gdf.rename(columns={'nom': 'nom_wil', 'code': 'code_wil'})
wilayas_gdf['centroid'] = wilayas_gdf['geometry'].centroid
wilayas_gdf.head()

<ipython-input-2-e95edb6e38e9>:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  wilayas_gdf['centroid'] = wilayas_gdf['geometry'].centroid


,code_wil,nom_wil,geometry,centroid
0,1,Adrar,"MULTIPOLYGON (((1.33461 20.72282, 1.32667 20.7...",POINT (-0.62174 26.01400)
1,2,Chlef,"MULTIPOLYGON (((1.55326 36.02990, 1.55325 36.0...",POINT (1.25669 36.23826)
2,3,Laghouat,"MULTIPOLYGON (((2.88464 32.88137, 2.87848 32.8...",POINT (2.72955 33.60757)
3,4,Oum El Bouaghi,"MULTIPOLYGON (((7.47320 35.52005, 7.47227 35.5...",POINT (7.06527 35.83646)
4,5,Batna,"MULTIPOLYGON (((6.10102 35.05459, 6.10090 35.0...",POINT (5.87455 35.41969)


In [4]:
wilayas_gdf = wilayas_gdf.set_crs("EPSG:4326")
wilayas_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [5]:
# Load Tagreted dataset and set to_check to False assuming all geocoded coordinates are OK 
pois_path = '../dz-datasets/agriculture/'
pois = pd.read_csv(pois_path + 'elfirma.csv')
pois['to_check'] = False
pois.head()

,Unnamed: 0,name,address,phone_number,category,organization,lat,lon,commune_name,wilaya_name,type_name,type_id,code_wil,to_check
0,0,Ouled Hamla SPA Tikouyavic,CITE GOUADJLIA RABAH PROJET 160 LOG BT 09,0798 22 14 13,pv,AGROLOG,36.027295,6.563820,AIN MLILA,OUM EL BOUAGHI,Viandes,1,4,False
1,1,Point de vente UPD Ain Kercha SPA Thagavic,cite Mohamed m'gena nord ouest Commune Ain Ke...,0561 65 30 18,pv,AGROLOG,35.928594,6.684329,AIN KERCHA,OUM EL BOUAGHI,Viandes,1,4,False
2,2,Point de vente kaous SPA TAHAGAVIC,cité stade Commune de Kaous,0661 33 60 29,pv,AGROLOG,36.377426,5.814824,KAOUS,JIJEL,Viandes,1,18,False
3,3,POINT DE VENTE DAWADJEN SITIFIS,RUE DES ABATTOIRS CITE LAID DHAOUI Sétif,0661 31 60 90,pv,AGROLOG,36.170960,5.406740,SETIF,SETIF,Viandes,1,19,False
4,4,POINT DE VENTE DAWADJEN SITIFIS,CITE BIR EN NSA AADL 3000 LOGTS LOCAL Num 124125,0770 80 45 32,pv,AGROLOG,36.176870,5.476410,SETIF,SETIF,Viandes,1,19,False


In [6]:
# Convert pandas DataFrame to GeoPandas 
pois['geometry'] = pois.apply(lambda row: Point(row.lon, row.lat), axis=1)
pois_gdf  = gpd.GeoDataFrame(pois)
pois_gdf.head()

,Unnamed: 0,name,address,phone_number,category,organization,lat,lon,commune_name,wilaya_name,type_name,type_id,code_wil,to_check,geometry
0,0,Ouled Hamla SPA Tikouyavic,CITE GOUADJLIA RABAH PROJET 160 LOG BT 09,0798 22 14 13,pv,AGROLOG,36.027295,6.563820,AIN MLILA,OUM EL BOUAGHI,Viandes,1,4,False,POINT (6.56382 36.02730)
1,1,Point de vente UPD Ain Kercha SPA Thagavic,cite Mohamed m'gena nord ouest Commune Ain Ke...,0561 65 30 18,pv,AGROLOG,35.928594,6.684329,AIN KERCHA,OUM EL BOUAGHI,Viandes,1,4,False,POINT (6.68433 35.92859)
2,2,Point de vente kaous SPA TAHAGAVIC,cité stade Commune de Kaous,0661 33 60 29,pv,AGROLOG,36.377426,5.814824,KAOUS,JIJEL,Viandes,1,18,False,POINT (5.81482 36.37743)
3,3,POINT DE VENTE DAWADJEN SITIFIS,RUE DES ABATTOIRS CITE LAID DHAOUI Sétif,0661 31 60 90,pv,AGROLOG,36.170960,5.406740,SETIF,SETIF,Viandes,1,19,False,POINT (5.40674 36.17096)
4,4,POINT DE VENTE DAWADJEN SITIFIS,CITE BIR EN NSA AADL 3000 LOGTS LOCAL Num 124125,0770 80 45 32,pv,AGROLOG,36.176870,5.476410,SETIF,SETIF,Viandes,1,19,False,POINT (5.47641 36.17687)


In [7]:
pois_gdf.count()

Unnamed: 0      486
name            486
address         486
phone_number     54
category        486
organization    486
lat             486
lon             486
commune_name    486
wilaya_name     486
type_name       486
type_id         486
code_wil        486
to_check        486
geometry        486
dtype: int64

In [8]:
# Set the Coordinate reference system
pois_gdf = pois_gdf.set_crs("EPSG:4326")
pois_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [9]:
# Check for rows with no coordiantes
no_coords_gdf = pois_gdf[(pois_gdf['lat'].isnull() | pois_gdf['lat'].isnull())]

# Merge with Wialyas DataFrame and set geometry to centroids
# The merge may be done either with Wilayas code or wilayas names   
no_coords_gdf = no_coords_gdf.merge(wilayas_gdf, left_on='code_wil', right_on='code_wil', how='left')
no_coords_gdf['geometry'] = no_coords_gdf['centroid']
no_coords_gdf['to_check'] = True

with_coords_gdf = no_coords_gdf.drop(columns=['geometry_x'], axis=1)

with_coords_gdf.count() 

Unnamed: 0      0
name            0
address         0
phone_number    0
category        0
organization    0
lat             0
lon             0
commune_name    0
wilaya_name     0
type_name       0
type_id         0
to_check        0
code_wil        0
nom_wil         0
geometry_y      0
centroid        0
geometry        0
dtype: int64

In [10]:
# Update the Targeted DataFrame by adding Wilayas geometries and centroids 
pois_gdf = pois_gdf[(pois_gdf['lat'].notnull() & pois_gdf['lat'].notnull())]

# Merge with Wialyas DataFrame and set geometry to centroids
# The merge may be done either with Wilayas code or wilayas names   
pois_gdf = pois_gdf.merge(wilayas_gdf, left_on='code_wil', right_on='code_wil', how='left')
pois_gdf.rename(columns={"geometry_x": "geometry"}, inplace=True)
pois_gdf.count()

Unnamed: 0      486
name            486
address         486
phone_number     54
category        486
organization    486
lat             486
lon             486
commune_name    486
wilaya_name     486
type_name       486
type_id         486
code_wil        486
to_check        486
geometry        486
nom_wil         486
geometry_y      486
centroid        486
dtype: int64

In [11]:
# Concat the Targeted DataFrame with rows without coordinates and approximated with Wilayas centroids 
pois_gdf = pd.concat([pois_gdf, with_coords_gdf])

# Drop unecessary columns
pois_gdf = pois_gdf.drop(columns=['centroid', 'nom_wil'], axis=1)

pois_gdf[(pois_gdf['lat'].isnull() | pois_gdf['lon'].isnull())]
pois_gdf.count()

Unnamed: 0      486
name            486
address         486
phone_number     54
category        486
organization    486
lat             486
lon             486
commune_name    486
wilaya_name     486
type_name       486
type_id         486
code_wil        486
to_check        486
geometry        486
geometry_y      486
dtype: int64

In [12]:
# Sort the DataFrame by code (Wilaya number) and reset the index
pois_gdf = pois_gdf.sort_values(by=['code_wil'],ignore_index=True)
pois_gdf.reset_index(drop=True)
pois_gdf.head()

,Unnamed: 0,name,address,phone_number,category,organization,lat,lon,commune_name,wilaya_name,type_name,type_id,code_wil,to_check,geometry,geometry_y
0,380,CCLS ADRAR,CCLS ADRAR,NaN,pv,OAIC,27.897786,-0.290856,ADRAR,ADRAR,Légumes secs,5,1,False,POINT (-0.29086 27.89779),"MULTIPOLYGON (((1.33461 20.72282, 1.32667 20.7..."
1,408,CCLS TENES,Station Ouled Farres RN 19 centre Ouled Fares,NaN,pv,OAIC,36.226154,1.234580,OULED FARES,CHLEF,Légumes secs,5,2,False,POINT (1.23458 36.22615),"MULTIPOLYGON (((1.55326 36.02990, 1.55325 36.0..."
2,410,CCLS TENES,Abou El Hassen Cite Abou Abou El Hassen,NaN,pv,OAIC,35.692364,4.533764,ABOU EL HASSEN,CHLEF,Légumes secs,5,2,False,POINT (4.53376 35.69236),"MULTIPOLYGON (((1.55326 36.02990, 1.55325 36.0..."
3,407,CCLS TENES,Station Oued Sly Cité Abdeli Oued Sly,NaN,pv,OAIC,36.106413,1.194841,OUED SLY,CHLEF,Légumes secs,5,2,False,POINT (1.19484 36.10641),"MULTIPOLYGON (((1.55326 36.02990, 1.55325 36.0..."
4,409,CCLS TENES,Dock Métal Chlef Route d'Alger Chlef,NaN,pv,OAIC,36.175738,1.354213,CHLEF,CHLEF,Légumes secs,5,2,False,POINT (1.35421 36.17574),"MULTIPOLYGON (((1.55326 36.02990, 1.55325 36.0..."


In [13]:
# Check if coordinates are within Wilayas polygons
pois_gdf['coords_in_wil'] = [pois_gdf.geometry[i].within(pois_gdf.geometry_y[i]) for i in range(pois_gdf.geometry.count())] 
pois_gdf[pois_gdf.coords_in_wil == False]

,Unnamed: 0,name,address,phone_number,category,organization,lat,lon,commune_name,wilaya_name,type_name,type_id,code_wil,to_check,geometry,geometry_y,coords_in_wil
2,410,CCLS TENES,Abou El Hassen Cite Abou Abou El Hassen,NaN,pv,OAIC,35.692364,4.533764,ABOU EL HASSEN,CHLEF,Légumes secs,5,2,False,POINT (4.53376 35.69236),"MULTIPOLYGON (((1.55326 36.02990, 1.55325 36.0...",False
94,42,OUED DJER,ROUTE NATIONAL Num 04 OUED DJER,0782 466 656,pv,AGROLOG,36.419722,2.396389,OUEDDJER,BLIDA,Viandes,1,9,False,POINT (2.39639 36.41972),"MULTIPOLYGON (((2.57442 36.37632, 2.57435 36.3...",False
110,379,LAITERIE NUMIDIA,Chef Lieu,NaN,pv,GIPLAIT,36.407832,8.112700,TEBESSA,TEBESSA,Produits laitiers,3,12,False,POINT (8.11270 36.40783),"MULTIPOLYGON (((7.36037 34.29662, 7.36034 34.2...",False
129,147,SOUK RAMADAN,Pose Num 10 El -Hamadia,035710168,pv,CNA,35.977007,4.748635,HAMADIA,TIARET,Fruits et légumes,2,14,False,POINT (4.74863 35.97701),"MULTIPOLYGON (((0.81448 34.61685, 0.81329 34.6...",False
197,375,EAC 5 BOURAADA (2ème point),EAC 5 BOURAADA (2ème point),NaN,pv,CNA,36.763618,3.368104,REGHAIA,ALGER,Fruits et légumes,2,16,False,POINT (3.36810 36.76362),"MULTIPOLYGON (((2.95544 36.61091, 2.95366 36.6...",False
212,374,EAC 5 BOURAADA,EAC 5 BOURAADA,NaN,pv,CNA,36.763618,3.368104,REGHAIA,ALGER,Fruits et légumes,2,16,False,POINT (3.36810 36.76362),"MULTIPOLYGON (((2.95544 36.61091, 2.95366 36.6...",False
227,2,Point de vente kaous SPA TAHAGAVIC,cité stade Commune de Kaous,0661 33 60 29,pv,AGROLOG,36.377426,5.814824,KAOUS,JIJEL,Viandes,1,18,False,POINT (5.81482 36.37743),"MULTIPOLYGON (((5.50709 36.59502, 5.50652 36.5...",False
317,11,PV 02 UPC BENBOULAID,VSA Mostagha benboulaid Ouad Seguene,0676 64 44 02,pv,AGROLOG,36.222812,6.517688,AIN SMARA,CONSTANTINE,Viandes,1,25,False,POINT (6.51769 36.22281),"MULTIPOLYGON (((6.53679 36.17123, 6.53731 36.1...",False
325,359,LAITERIE NUMIDIA,Centre ville (Jijel),NaN,pv,GIPLAIT,36.810630,5.762748,CONSTANTINE,CONSTANTINE,Produits laitiers,3,25,False,POINT (5.76275 36.81063),"MULTIPOLYGON (((6.53679 36.17123, 6.53731 36.1...",False
343,247,LAITERIELE LITTORAL,Site de la laiterie Salamandre,NaN,pv,GIPLAIT,35.100000,0.069000,MOSTAGANEM,MOSTAGANEM,Produits laitiers,3,27,False,POINT (0.06900 35.10000),"MULTIPOLYGON (((0.01588 35.74168, 0.01562 35.7...",False


In [14]:
# Update to_check flag to take into account points outside Wilays and without geocoding coordinates
final_gdf = pois_gdf
final_gdf['to_check'] = (pois_gdf['to_check'] == True) | (pois_gdf['coords_in_wil'] == False)   

# Remove unecessary columns
final_gdf = final_gdf.drop(['geometry_y', 'coords_in_wil'], axis=1)

final_gdf[final_gdf.to_check == True]

,Unnamed: 0,name,address,phone_number,category,organization,lat,lon,commune_name,wilaya_name,type_name,type_id,code_wil,to_check,geometry
2,410,CCLS TENES,Abou El Hassen Cite Abou Abou El Hassen,NaN,pv,OAIC,35.692364,4.533764,ABOU EL HASSEN,CHLEF,Légumes secs,5,2,True,POINT (4.53376 35.69236)
94,42,OUED DJER,ROUTE NATIONAL Num 04 OUED DJER,0782 466 656,pv,AGROLOG,36.419722,2.396389,OUEDDJER,BLIDA,Viandes,1,9,True,POINT (2.39639 36.41972)
110,379,LAITERIE NUMIDIA,Chef Lieu,NaN,pv,GIPLAIT,36.407832,8.112700,TEBESSA,TEBESSA,Produits laitiers,3,12,True,POINT (8.11270 36.40783)
129,147,SOUK RAMADAN,Pose Num 10 El -Hamadia,035710168,pv,CNA,35.977007,4.748635,HAMADIA,TIARET,Fruits et légumes,2,14,True,POINT (4.74863 35.97701)
197,375,EAC 5 BOURAADA (2ème point),EAC 5 BOURAADA (2ème point),NaN,pv,CNA,36.763618,3.368104,REGHAIA,ALGER,Fruits et légumes,2,16,True,POINT (3.36810 36.76362)
212,374,EAC 5 BOURAADA,EAC 5 BOURAADA,NaN,pv,CNA,36.763618,3.368104,REGHAIA,ALGER,Fruits et légumes,2,16,True,POINT (3.36810 36.76362)
227,2,Point de vente kaous SPA TAHAGAVIC,cité stade Commune de Kaous,0661 33 60 29,pv,AGROLOG,36.377426,5.814824,KAOUS,JIJEL,Viandes,1,18,True,POINT (5.81482 36.37743)
317,11,PV 02 UPC BENBOULAID,VSA Mostagha benboulaid Ouad Seguene,0676 64 44 02,pv,AGROLOG,36.222812,6.517688,AIN SMARA,CONSTANTINE,Viandes,1,25,True,POINT (6.51769 36.22281)
325,359,LAITERIE NUMIDIA,Centre ville (Jijel),NaN,pv,GIPLAIT,36.810630,5.762748,CONSTANTINE,CONSTANTINE,Produits laitiers,3,25,True,POINT (5.76275 36.81063)
343,247,LAITERIELE LITTORAL,Site de la laiterie Salamandre,NaN,pv,GIPLAIT,35.100000,0.069000,MOSTAGANEM,MOSTAGANEM,Produits laitiers,3,27,True,POINT (0.06900 35.10000)


In [15]:
final_gdf.count()

Unnamed: 0      486
name            486
address         486
phone_number     54
category        486
organization    486
lat             486
lon             486
commune_name    486
wilaya_name     486
type_name       486
type_id         486
code_wil        486
to_check        486
geometry        486
dtype: int64

In [16]:
# Write to a GeoJSON file

gpd.GeoDataFrame(final_gdf).to_file(pois_path + "pois_checked_in_out_wilayas.geojson", driver='GeoJSON')